In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## check:
    ### CLE base in GAINS or MLE??
    ### More pulses in more places

In [3]:
# found here: https://gains.iiasa.ac.at/gains/reporting/select?versionID=ASN&rtype=datatable#
# based on the emissions factors from publication: 

#go to advanced model Asia, click all sources under emissions factors, select relevant details using ECLIPSE v6 and PM_BC, 
#other choice needed is the country and I think CLE base


## CECILIA HAN SPRINGER, ERL PAPER

In [4]:
df = pd.read_csv('Springer_ERL/coal-plant-data.csv', encoding = 'ISO-8859-1')

In [5]:
### you need to do pulses for more places #######

In [6]:
df.loc[(df['AE.CHINA'] == True)].groupby('COUNTRY').count()

,UNITID,UNIT,PLANT,COMPANY,MW,STATUS,YEAR,UTYPE,FUEL,FUELTYPE,...,BUSTYPE,COMPID,LOCATIONID,HEATRATE.UNADJ,EMISFACTOR.PLATTS,CAPFACTOR,ANNUALCO2,EMISFACTOR.GCPT,HEATRATE.ADJ,COAL
COUNTRY,,,,,,,,,,,,,,,,,,,,,
BANGLADESH,13,13,13,13,13,13,8,13,13,5,...,13,13,13,9,1,13,1,13,7,5
CAMBODIA,6,6,6,6,6,6,5,6,6,0,...,6,6,6,5,0,6,0,6,5,0
CHINA,1599,1599,1599,1599,1599,1599,1445,1599,1599,1119,...,1599,1599,1599,1595,1267,1599,1185,1497,1476,1119
INDIA,152,152,152,152,152,152,121,152,152,80,...,152,152,152,150,112,152,99,140,132,80
INDONESIA,132,132,132,132,132,132,109,132,132,34,...,132,132,132,126,61,132,49,94,111,34
JAPAN,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
LAOS,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,0,3,0,3,3,3
MALAYSIA,13,13,13,13,13,13,9,13,13,9,...,13,13,13,13,9,13,9,9,9,9
MONGOLIA,17,17,17,17,17,17,8,17,17,12,...,17,17,17,15,10,17,6,13,8,12


In [7]:
df_test_set = df.loc[(df['AE.CHINA'] == True) & (df['COUNTRY'].isin(['INDONESIA','CAMBODIA', 'MALAYSIA', 'VIETNAM']))]

## GAINS

In [8]:
df_dict = {}

In [9]:
for idx in [f' ({i})' for i in np.arange(1,11)]:
    df_gains = pd.read_excel(f'GAINS_emis_factors/emiss_factors{idx}.xlsx', header = [0,7])
    df_gains = df_gains.drop(0)
    df_gains.columns = df_gains.columns.droplevel(0)
    df_gains = df_gains.drop(columns = ['Emission factor [kt/act.unit].1','Emission factor [kt/act.unit]', 'kt PM PM_BC/Act.unit','Technology.1'])
    df_gains = df_gains.rename(columns = {'Technology':'Emission factor [kt/act.unit]', 
                 'Sector.1': 'kt PM PM_BC/Act.unit',
                 'Sector': 'Technology',
                 'Fuel Activity.1':'Sector'})
    df_dict[df_gains.iloc[-1]['Fuel Activity']] = df_gains

In [10]:
df_EF = pd.DataFrame(columns = ['PP_MOD_EF'])

for r in df_dict.keys(): ## using PP MOD, but should look into differentiation by type
    df_EF.loc[r] = df_dict[r].loc[df_dict[r]['Sector'] == 'PP_MOD']['Emission factor [kt/act.unit]'].mean()

In [11]:
df_EF_final = {'INDONESIA':np.mean([df_EF.loc['INDONESIA_Sumatra'].values[0],df_EF.loc['INDONESIA_Rest of country'].values[0],
                                   df_EF.loc['INDONESIA_Java'].values[0],df_EF.loc['INDONESIA_Jakarta'].values[0]]),
              'MALAYSIA': np.mean([df_EF.loc['MALAYSIA_Sarawak-Sabah'].values[0],df_EF.loc['MALAYSIA_Peninsular Malaysia'].values[0],
                                   df_EF.loc['MALAYSIA_Kuala Lumpur'].values[0]]),
              'VIETNAM': np.mean([df_EF.loc['VIETNAM_South: Mekong River Delta-Ho Chi Minh City'].values[0],
                                   df_EF.loc['VIETNAM_North: Red River Delta-Hanoi'].values[0]]),
              'CAMBODIA': np.mean([df_EF.loc['CAMBODIA'].values[0]])}

In [12]:
df_EF_final #kt/act unit wherre act unit is PJ
for c in df_EF_final.keys():
    df_EF_final[c]/=1e-9 #kt/MW

In [13]:
for c in df_EF_final.keys():
    df_test_set.loc[df_test_set['COUNTRY'] == c, 'EMISFACTOR.BC.GAINS'] = df_EF_final[c]

/home/emfreese/anaconda3/envs/grid_mod/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/emfreese/anaconda3/envs/grid_mod/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
## 1 PJ = 277777.7777777778 MWh
PJ_to_MWh = 1/277777.7777777778 
## 1 kt = 1000000000 g
kt_to_g = 1/1000000000
MW_to_MWh= 24 #daily

In [15]:
##### CHECK CALCULATION FOR BC KT TO G ##########

In [16]:
df_test_set.loc[:,'EMISFACTOR.BC.GAINS.g_mwh'] = df_test_set.loc[:,'EMISFACTOR.BC.GAINS']* PJ_to_MWh #kt/MWh
df_test_set.loc[:,'EMISFACTOR.BC.GAINS.g_mwh'] = df_test_set.loc[:,'EMISFACTOR.BC.GAINS']* kt_to_g #g/MWh 


/home/emfreese/anaconda3/envs/grid_mod/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/emfreese/anaconda3/envs/grid_mod/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [17]:
df_test_set.loc[:,'MWh_daily'] = df_test_set['MW']*MW_to_MWh

In [18]:
df_test_set.loc[:,'BC (g/day)'] = df_test_set['MWh_daily']*df_test_set['EMISFACTOR.BC.GAINS.g_mwh']

In [19]:
df_test_set.to_csv('mod_coal_inputs/BC_limited_country_SEA_GAINS_Springer.csv')